이번 Lab을 통해 XOR 문제를 다층 퍼셉트론으로 구현해보겠습니다!

In [1]:
import torch
import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

XOR 문제를 풀기 위한 입력과 출력을 정의해줍니다.

In [3]:
X = torch.FloatTensor([[0,0],[0,1], [1,0], [1,1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

입력층, 은닉층 3개, 출력층을 가지는 인공신경망을 구현해보겠습니다.

In [4]:
model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input layer의 크기를 생각해보세요.
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), 
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), 
          nn.Sigmoid(),
          nn.Linear(10,1,bias=True), # output layer의 크기를 생각해보세요.
          nn.Sigmoid()
          ).to(device)

In [5]:
"""
# 이렇게 구현해도 됩니다.
# nn layers
linear1 = torch.nn.Linear(2, 10, bias=True) # input layer의 크기를 생각해보세요.
linear2 = torch.nn.Linear(10, 10, bias=True)
linear3 = torch.nn.Linear(10, 10, bias=True)
linear4 = torch.nn.Linear(10,1,bias=True) # output layer의 크기를 생각해보세요.
sigmoid = torch.nn.Sigmoid()

# model
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)
"""

'\n# 이렇게 구현해도 됩니다.\n# nn layers\nlinear1 = torch.nn.Linear(2, 10, bias=True) # input layer의 크기를 생각해보세요.\nlinear2 = torch.nn.Linear(10, 10, bias=True)\nlinear3 = torch.nn.Linear(10, 10, bias=True)\nlinear4 = torch.nn.Linear(10,1,bias=True) # output layer의 크기를 생각해보세요.\nsigmoid = torch.nn.Sigmoid()\n\n# model\nmodel = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)\n'

이제 비용 함수와 옵타마이저를 선언합니다.

In [7]:
# define cost/loss & optimizer
criterion = torch.nn.BCELoss().to(device) # 이진 분류에서 사용하는 크로스엔트로피 함수를 사용하세요.
optimizer = torch.optim.SGD(model.parameters(), lr = 1) # SGD optimizer를 사용하고 learing rate는 1로 설정하세요.

총 10,001번의 에포크를 수행합니다. 각 에포크마다 역전파가 수행된다고 보면 되겠습니다.

In [8]:
for step in range(10001):
    optimizer.zero_grad()
    #이 부분을 채워넣으세요.#  # gradient를 0으로 초기화
    hypothesis = model(X) # 모델을 생성해서 hypothesis를 구해보세요.

    # cost/loss function
    cost = criterion(hypothesis, Y) # 위에서 정의한 비용함수를 사용하세요.
    #이 부분을 채워넣으세요.#  # 비용 함수를 미분하여 gradient 계산
    #이 부분을 채워넣으세요.#  # W와 b를 업데이트
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item())

0 0.6948983669281006
100 0.693155825138092
200 0.6931535601615906
300 0.6931513547897339
400 0.693149209022522
500 0.6931473016738892
600 0.6931453943252563
700 0.6931434273719788
800 0.6931416988372803
900 0.6931397914886475
1000 0.6931380033493042
1100 0.6931361556053162
1200 0.6931343078613281
1300 0.6931324005126953
1400 0.6931304931640625
1500 0.6931284666061401
1600 0.6931264400482178
1700 0.6931242942810059
1800 0.6931220889091492
1900 0.6931197047233582
2000 0.6931171417236328
2100 0.6931145191192627
2200 0.6931115984916687
2300 0.6931084990501404
2400 0.6931051015853882
2500 0.6931014657020569
2600 0.6930974721908569
2700 0.6930930018424988
2800 0.6930879950523376
2900 0.6930825114250183
3000 0.6930763721466064
3100 0.6930692195892334
3200 0.6930612325668335
3300 0.6930519342422485
3400 0.6930410861968994
3500 0.6930283904075623
3600 0.6930132508277893
3700 0.6929951310157776
3800 0.6929728984832764
3900 0.6929453015327454
4000 0.6929103136062622
4100 0.6928649544715881
4200 0

이제 모델이 XOR 문제를 풀 수 있는지 테스트 해봅시다.

In [9]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[1.11739784e-04]
 [9.99828696e-01]
 [9.99842167e-01]
 [1.85383164e-04]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


실제값은 0, 1, 1, 0이며 예측값은 0, 1, 1, 0으로 문제를 해결하는 모습을 보여줍니다.

Q1. Backpropagation이란?

=> 실제값과 예측값이 얼마나 차이가 나는지 구한 후 오차값을 다시 뒤로 가면서 변수 갱신.

Q2. Backpropagation은 어떤 방식을 이용해 weight, bias 같은 변수들을 업데이트 할까요?

=> Chain rule 이용해서 업데이트.